Insert explanatory text here.

In [1]:
load("../Shared/weil_poly_utils.sage")
load("../Shared/cyclic_covers.sage")

Tabulate the possible point counts for the curve corresponding to $F$.

In [2]:
import pandas

In [3]:
df = pandas.read_excel('../Shared/polys.xlsx')
assert 'Cyclic' in df
print(list(df))

['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'd', 'g', "g'", 'Label of J(C)', '#J(C)(F_2)', '#J(C)(F_4)', 'Counts of C', "Counts of C'", 'Cyclic']


In [4]:
candidates = {}
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    if (d,g,g1) not in candidates:
        candidates[d,g,g1] = []
    counts1 = tuple(eval(r["Counts of C"]))
    candidates[d,g,g1].append(counts1)

In [5]:
targets = [t[:5] for t in candidates[2, 5, 9]]

Read Dragutinovic's data on hyperelliptic curves, then compare their Frobenius polynomials to `targets`.

In [6]:
F = GF(2)
P.<x> = F[]
with open("HyperellipticCurvesData.txt", "r") as f:
    s = f.read()
    l = sage_eval(s, locals={'x':x})
len(l)

1070

In [7]:
Q.<T> = QQ[]
for (h, f) in l:
    C = HyperellipticCurve(f,h)
    assert C.genus() == 5
    frob = C.frobenius_polynomial()(T)
    ct = point_count_from_weil_poly(frob, 5)
    assert ct not in targets

Read Dragutinovic's data on trigonal curves, then compare their Frobenius polynomials to `targets`.

In [8]:
with open("TrigonalCurvesWithAutomorphisms.txt", "r") as f:
    s = f.read()
    l = sage_eval(s)
len(l)

2854

In [9]:
P.<x0,x1,x2> = F[]
monos5 = [prod(t) for t in itertools.combinations_with_replacement(P.gens(), 5)]
len(monos5)

21

In [10]:
proj = magma.ProjectiveSpace(P)
l2 = []
for (v, _) in l:
    gen = sum(v[i]*monos5[i] for i in range(len(monos5)))
    X = proj.Scheme([gen])
    assert X.Dimension() == 1
    C = X.Curve()
    assert C.Genus() == 5
    ct = tuple(Integer(C.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, 6))
    if ct in targets:
        l2.append(C)

In [11]:
len(l2)

73

Compute cyclic covers of the resulting curves. This requires a conversion between Magma's category of function fields associated to schemes and Magma's category of algebraic function fields (i.e., finite extensions of a univariate rational function field).

In [12]:
l3 = []
for C in l2:
    F0 = C.FunctionField()
    F1 = F0.AlgorithmicFunctionField()
    l3 = l3 + match_weil_poly(F1, 2)

In [13]:
assert not l3

In [14]:
print("All tests completed!")

All tests completed!
